In [2]:
from sklearn import svm, datasets
iris = datasets.load_iris()

In [4]:
import pandas as pd
df = pd.DataFrame(iris.data, columns = iris.feature_names)
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2)

In [9]:
model = svm.SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.9666666666666667

In [10]:
from sklearn.model_selection import cross_val_score

In [11]:
cross_val_score(svm.SVC(kernel='linear', C=10, gamma='auto'), iris.data, iris.target, cv=5) # 5-fold

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [12]:
cross_val_score(svm.SVC(kernel='rbf', C=10, gamma='auto'), iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [13]:
cross_val_score(svm.SVC(kernel='rbf', C=20, gamma='auto'), iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [20]:
# now this permutation and combination is very lengthy
# so using for loop

import numpy as np
kernels = ['rbf','linear']
C = [1,2,3,10,20,15]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval, C=cval, gamma='auto'), iris.data, iris.target, cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)
        
avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_2': 0.9800000000000001,
 'rbf_3': 0.9733333333333334,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'rbf_15': 0.9733333333333334,
 'linear_1': 0.9800000000000001,
 'linear_2': 0.9800000000000001,
 'linear_3': 0.9733333333333334,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666,
 'linear_15': 0.9666666666666666}

In [36]:
# now GridSearchCV does this exact same thing using k-fold validation

from sklearn.model_selection import GridSearchCV

classifier = GridSearchCV(svm.SVC(gamma='auto'), cv=5, return_train_score=False
                         ,param_grid={'C':[1,2,3,10,20,15],'kernel': ['rbf', 'linear']})

classifier.fit(iris.data, iris.target)
classifier.cv_results_

{'mean_fit_time': array([0.00147023, 0.00125761, 0.00196319, 0.0018137 , 0.00096793,
        0.00092716, 0.00098104, 0.00085998, 0.00098653, 0.0009078 ,
        0.00093007, 0.00089498]),
 'std_fit_time': array([4.57046056e-04, 2.39026078e-04, 5.20359927e-04, 7.28466241e-04,
        5.06082172e-05, 9.83516837e-05, 1.01666155e-04, 6.67757622e-05,
        1.11796950e-04, 1.21254036e-04, 3.32725065e-05, 8.96437878e-05]),
 'mean_score_time': array([0.00105782, 0.00099626, 0.00206237, 0.00098982, 0.00057344,
        0.00050421, 0.00053802, 0.00053263, 0.00054741, 0.00047331,
        0.00054474, 0.00048475]),
 'std_score_time': array([3.89336310e-04, 3.34664300e-04, 7.87979881e-04, 4.61824323e-04,
        2.58450139e-05, 4.66100456e-05, 1.43808901e-05, 8.31324252e-05,
        2.64745556e-05, 2.08113063e-05, 2.14647705e-05, 4.29257804e-05]),
 'param_C': masked_array(data=[1, 1, 2, 2, 3, 3, 10, 10, 20, 20, 15, 15],
              mask=[False, False, False, False, False, False, False, False,
    

In [37]:
# exporting this result in form of pandas df

df = pd.DataFrame(classifier.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001470,0.000457,0.001058,0.000389,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001258,0.000239,0.000996,0.000335,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001963,0.000520,0.002062,0.000788,2,rbf,"{'C': 2, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.001814,0.000728,0.000990,0.000462,2,linear,"{'C': 2, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
4,0.000968,0.000051,0.000573,0.000026,3,rbf,"{'C': 3, 'kernel': 'rbf'}",0.966667,1.0,0.933333,0.966667,1.0,0.973333,0.024944,6
5,0.000927,0.000098,0.000504,0.000047,3,linear,"{'C': 3, 'kernel': 'linear'}",0.966667,1.0,0.933333,0.966667,1.0,0.973333,0.024944,6
6,0.000981,0.000102,0.000538,0.000014,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
7,0.000860,0.000067,0.000533,0.000083,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,6
8,0.000987,0.000112,0.000547,0.000026,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,10
9,0.000908,0.000121,0.000473,0.000021,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,11


In [53]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,2,rbf,0.980000
3,2,linear,0.980000
4,3,rbf,0.973333
5,3,linear,0.973333
6,10,rbf,0.980000
7,10,linear,0.973333
8,20,rbf,0.966667
9,20,linear,0.966667


In [39]:
dir(classifier)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 '_validate_params',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'inverse_transform',
 'multimetric_',
 'n_fe

In [40]:
classifier.best_score_

0.9800000000000001

In [42]:
classifier.best_params_

{'C': 1, 'kernel': 'rbf'}

In [43]:
# this is a very high computation method as it tries every permutation and cobination for param values
# imagine if you give a range of 1 to 50 to be supplied as value of the grid

In [44]:
# to tackle this computation problem sklearn has Randomised SearchCV
# this will not try each and every value but it will try random values for the params and you can choose what those iteration can be

In [48]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
        'C':[1,2,3,10,20,15],
        'kernel':['rbf','linear']
    },
    cv=5, return_train_score=False, n_iter=2 #it will try any of the 2 combination
)

In [50]:
rs.fit(iris.data, iris.target)
pd.DataFrame(rs.cv_results_)[['param_C', 'param_kernel', 'mean_test_score']]

# this works well in practical life

,param_C,param_kernel,mean_test_score
0,20,linear,0.966667
1,10,linear,0.973333


In [51]:
# how to choose best model for given data

In [52]:
# different models with different params

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [60]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 10}
2,logistic_regression,0.966667,{'C': 5}
